# Outline
0. Discrete space setup
1. Discrete Ergodic Coverage
2. Maximizing Information Gathering

In [33]:
distribution_name = "paper_peaks" # "close_peaks"
time_steps = 100
K = 5
system1_name = "casadi"
system2_name = "ig"
compute_mu = False
grid_length = 10

dx = 0.001

In [2]:
import numpy as np
U_shape = (1,1)
all_k = list(np.ndindex(*[K]*len(U_shape)))

# 0. Discrete Space Setup

## 0.a. Define probability distribution $\mu$

In [3]:
# defining probability distribution mu
from probability_distribution import *
mu_unnorm = mu_gaussians([(np.array([0.25, 0.75]), 0.025), (np.array([0.6, 0.4]), 0.025)], U_shape) # paper peaks
# mu_unnorm = mu_gaussians([(np.array([0.25, 0.75]), 0.025), (np.array([0.5, 0.5]), 0.025)], U_shape)  # close peaks
total = mu_total(mu_unnorm, U_shape)
# mu = mu_gaussians([(np.array([0.2, 0.75]), 0.1)], U_shape)
mu = lambda x: mu_unnorm(x)/total # mu_normalize(mu_unnorm, U_shape)

# calculating fourier coefficients of probability distribution mu
from fourier_functions import *
import pickle

ff = Fourier_Functions(mu, U_shape, K, compute_mu=compute_mu)
if compute_mu:
    mu_k = {}
    for k in ff:
        mu_k[k] = ff[k]['mu_k']

    with open(f'mu/{distribution_name}_{K}.pkl', 'wb') as handle:
        pickle.dump(mu_k, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f'mu/{distribution_name}_{K}.pkl', 'rb') as handle:
        mu_k = pickle.load(handle)
    for k in ff:
        ff[k]['mu_k'] = mu_k[k]

original_mu = mu 
# mu = fourier_coefficient2distribution(ff, all_k, c_k=None)


## 1. Generate Vanilla Ergodic Trajectory $x$

In [19]:
mu_display2D(original_mu, U_shape, f"mu/{distribution_name}_original.pdf")

from ergodic_agents import *
from mm_agent import *
from casadi_agent import *

agent1 = CasadiAgent(0, np.array([0.2, 0.3]), 0.5, all_k, U_shape, ff, eps=1e-5)
system1 = AgentSystem([agent1], mu, U_shape, ff, K)
agent1.time_horizon = 30

In [20]:
t = 0
delta_t = 0.1
for i in range(time_steps):
    t = i * delta_t
    system1.evolve(t, delta_t)


      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  26.19ms (  1.05ms)  26.28ms (  1.05ms)        25
       nlp_g  |   1.22ms ( 48.88us)   1.22ms ( 48.78us)        25
  nlp_grad_f  |  47.19ms (  1.81ms)  47.30ms (  1.82ms)        26
  nlp_hess_l  |   5.50 s (229.30ms)   5.50 s (229.30ms)        24
   nlp_jac_g  |   6.31ms (242.81us)   6.33ms (243.62us)        26
       total  |   5.63 s (  5.63 s)   5.63 s (  5.63 s)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  26.25ms (  1.01ms)  26.32ms (  1.01ms)        26
       nlp_g  |   1.15ms ( 44.31us)   1.15ms ( 44.16us)        26
  nlp_grad_f  |  44.15ms (  1.84ms)  44.26ms (  1.84ms)        24
  nlp_hess_l  |   5.21 s (237.02ms)   5.21 s (237.03ms)        22
   nlp_jac_g  |   5.65ms (235.25us)   5.67ms (236.37us)        24
       total  |   5.33 s (  5.33 s)   5.33 s (  5.33 s)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp

In [21]:
filename1 = f"optimality/system={system1_name}_mu={distribution_name}_K={K}_T={time_steps}_time_horizon={agent1.time_horizon}"
description = f"{system1_name} on {distribution_name}"
system1.mu = original_mu
system1.visualize_trajectory(filename1, description)
system1.visualize_ergodicity(filename1)

## 2. Maximize Information Gathering

In [23]:
_mu_casadi = mu_gaussians_casadi([(np.array([0.25, 0.75]), 0.025), (np.array([0.6, 0.4]), 0.025)], U_shape) # paper peaks
# _mu_casadi = mu_gaussians_casadi([(np.array([0.25, 0.75]), 0.025), (np.array([0.5, 0.5]), 0.025)], U_shape) # close peaks

mu_casadi = lambda x: _mu_casadi(x)/total

In [34]:
from casadi_agent_info_gathering import *
agent2 = CasadiAgentInfoGathering(0, np.array([0.2, 0.3]), 0.5, all_k, U_shape, ff, eps=1e-5)
agent2.mu = mu_casadi #lambda x: casadi.norm_2(x) # need to do gaussian but for casadi
agent2.dx = dx
system2 = AgentSystem([agent2], mu_casadi, U_shape, ff, K)

In [26]:
print(agent2.x_log[0].shape)

(2,)


In [35]:
t = 0
delta_t = 0.1
for i in range(time_steps):
    t = i * delta_t
    system2.evolve(t, delta_t)

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   4.08 s (  7.63ms)   4.08 s (  7.63ms)       535
       nlp_g  |  59.88ms (111.92us)  59.47ms (111.17us)       535
  nlp_grad_f  |   7.76 s ( 17.48ms)   7.76 s ( 17.49ms)       444
  nlp_hess_l  | 860.32 s (  1.95 s) 860.33 s (  1.95 s)       442
   nlp_jac_g  | 579.51ms (  1.31ms) 581.35ms (  1.31ms)       444
       total  | 889.44 s (889.44 s) 889.44 s (889.44 s)         1


In [36]:
filename2 = f"optimality/system={system2_name}_mu={distribution_name}_dx={dx}_T={time_steps}"
description = f"{system2_name} on {distribution_name}"

system2.visualize_trajectory(filename2, description)
# system2.visualize2d(filename=filename2, additional_title=description, plot_c_k=False)
system2.visualize_ergodicity(filename2)

## 3. Comparison

In [ ]:
results = {}

results['ergodic_trajectory'] = {}
results['ergodic_trajectory']['trajectory'] = np.array(agent1.x_log)
results['ergodic_trajectory']['ergodicity'] = agent1.e_log
results['ergodic_trajectory']['info gathered'] = [info_gathering_metric_noncasadi(mu, agent1.x_log, T+1, 1/dx) for T in range(time_steps)]

results['info_gather_trajectory'] = {}
results['info_gather_trajectory']['trajectory'] = np.array(agent2.x_log)
results['info_gather_trajectory']['ergodicity'] = np.array(agent2.e_log)
results['info_gather_trajectory']['info gathered'] = np.array([info_gathering_metric(mu_casadi, agent2.x_log, T+1, 1/dx, T+1) for T in range(time_steps)]).reshape(-1,)


In [ ]:
filename = f"optimality/system={system1_name}_{system2_name}_mu={distribution_name}_K={K}_dx={dx}_T={time_steps}"

with open(f'{filename}_results.pkl', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

def plot_trajectory_values(results, filename, trajectory_name):
    plt.figure()
    plt.plot(results[trajectory_name]['ergodicity'][1:], results[trajectory_name]['info gathered'], 'o')
    plt.xlabel("Ergodicity")
    plt.xscale("log")
    plt.gca().invert_xaxis()
    plt.ylabel("Information Gathered")
    plt.savefig(f"{filename}_ergodicity_vs_info_gathered.pdf")
plot_trajectory_values(results, filename1, 'ergodic_trajectory')
plot_trajectory_values(results, filename2, 'info_gather_trajectory')

trajectories = [results['ergodic_trajectory']['trajectory'], results['info_gather_trajectory']['trajectory']]
visualize_trajectory(f"{filename}_ergodicity_vs_info_gathered", 
                     f"Ergodic vs Info Gathering Trajectory", 
                     U_shape, trajectories, mu)


In [ ]:
for trajectory in ['ergodic_trajectory', 'info_gather_trajectory']:
    print("Trajectory: ", trajectory)
    print("Final ergodicity: ", results[trajectory]['ergodicity'][-1])
    print("Final Info Gather: ", results[trajectory]['info gathered'][-1])

Trajectory:  ergodic_trajectory
Final ergodicity:  1.939733715033913e-08
Final Info Gather:  3.6086141608097564
Trajectory:  info_gather_trajectory
Final ergodicity:  0.024780407724141654
Final Info Gather:  -8.171378168402923
